In [ ]:
import timeit

In [1]:
import os
os.chdir('../../')
!pwd

/root/python/myenv/medical-coding-reproducibility-main


In [2]:
import logging
import math
import os
from pathlib import Path

from hydra import initialize, compose
from rich.pretty import pprint
from hydra.core.global_hydra import GlobalHydra

from src.data.data_pipeline import data_pipeline
from src.factories import (
    get_callbacks,
    get_dataloaders,
    get_datasets,
    get_lookups,
    get_lr_scheduler,
    get_metric_collections,
    get_model,
    get_optimizer,
    get_text_encoder,
    get_transform,
)
from src.trainer.trainer import Trainer
from src.utils.seed import set_seed



LOGGER = logging.getLogger(name='test')
LOGGER.setLevel(logging.INFO)



def deterministic() -> None:
    """Run experiment deterministically. There will still be some randomness in the backward pass of the model."""
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

    import torch

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(True)


# Clear the global Hydra instance
GlobalHydra.instance().clear()
#Load configuration


initialize(config_path="../../configs")
#caml multi_res_conv vanillaconv vanillagru laat plm_icd
cfg = compose(config_name="config",
              overrides=["experiment=mimiciv_icd10/plm_icd.yaml",
                         "callbacks=no_wandb",
                         "load_model=null",
                         "trainer.epochs=0"]
                          )
mapping_model = {
    "CAML": "./experiments/jkz0ogtz",
    "PLMICD": "./experiments/jfywyfod",
    "LAAT": "./experiments/gsycv1tg",
    "MultiResCNN": "./experiments/t1dbhfub",
    "VanillaRNN": "./experiments/mjukdu9c",
    "VanillaConv": "./experiments/vy559dlj",
}

cfg.load_model = mapping_model[cfg.model.name]

if cfg.deterministic:
    deterministic()
else:
    import torch

    if torch.cuda.is_available():
        print("GPU is available")
        print(f"GPU name: {torch.cuda.get_device_name(0)}")
    else:
        print("GPU is not available")
set_seed(cfg.seed)


# Check if CUDA_VISIBLE_DEVICES is set
if "CUDA_VISIBLE_DEVICES" not in os.environ:
    if cfg.gpu != -1 and cfg.gpu is not None and cfg.gpu != "":
        os.environ["CUDA_VISIBLE_DEVICES"] = (
            ",".join([str(gpu) for gpu in cfg.gpu])
            if isinstance(cfg.gpu, list)
            else str(cfg.gpu)
        )

    else:
        os.environ["CUDA_VISIBLE_DEVICES"] = ""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pprint(f"Device: {device}")
pprint(f"CUDA_VISIBLE_DEVICES: {os.environ['CUDA_VISIBLE_DEVICES']}")

/tmp/ipykernel_87750/1143752983.py:49: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../../configs")


GPU is not available


'Device: cpu'

'CUDA_VISIBLE_DEVICES: [6]'

In [3]:
from pprint import pprint

pprint(cfg)


{'seed': 1337, 'deterministic': False, 'gpu': [6], 'name': None, 'debug': False, 'load_model': './experiments/jfywyfod', 'data': {'dir': 'files/data/mimiciv_icd10', 'data_filename': 'mimiciv_icd10.feather', 'split_filename': 'mimiciv_icd10_split.feather', 'code_column_names': ['icd10_diag', 'icd10_proc'], 'max_length': 4000}, 'dataset': {'name': 'HuggingfaceDataset', 'configs': {'chunk_size': 128}}, 'dataloader': {'max_batch_size': 1, 'batch_size': 16, 'num_workers': 0, 'drop_last': True, 'pin_memory': False, 'batch_sampler': {'name': 'BySequenceLengthSampler', 'configs': {'bucket_boundaries': [400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2600, 3000, 4000]}}}, 'model': {'name': 'PLMICD', 'configs': {'model_path': './files/hfs/RoBERTa-base-PM-M3-Voc-hf'}}, 'text_encoder': {}, 'trainer': {'epochs': 0, 'validate_on_training_data': True, 'print_metrics': False, 'use_amp': True, 'threshold_tuning': True}, 'optimizer': {'name': 'AdamW', 'configs': {'lr': 5e-05, 'weight_decay': 0}

พูดถึง vaex

'split_filename': 'mimiciv_icd10_split.feather'

In [4]:
data = data_pipeline(config=cfg.data)

In [5]:
text_encoder = get_text_encoder(
    config=cfg.text_encoder, data_dir=cfg.data.dir, texts=data.get_train_documents
)

ตรงนี้เปลี่ยน target เป็น int นะ 

In [6]:
label_transform = get_transform(
    config=cfg.label_transform,
    targets=data.all_targets,
    load_transform_path=cfg.load_model,
)

loaded transform


set TokenSequence, HuggingFaceTokenizer, BOW แต่งเติมส่วนของ padding unknow ให้เรียบร้อย

In [7]:
text_transform = get_transform(
    config=cfg.text_transform,
    texts=data.get_train_documents,
    text_encoder=text_encoder,
    load_transform_path=cfg.load_model,
)

./files/hfs/RoBERTa-base-PM-M3-Voc-hf
loaded transform


Tranform text จริง โดยมองเป็น batch

In [8]:
data.truncate_text(cfg.data.max_length)

In [9]:
#transform token=>index ตัวเลข เด๋ว train จะถูกผลักออกเป็น vector 
data.transform_text(text_transform.batch_transform)

Transforming text...: 100%|██████████| 31/31 [00:02<00:00, 14.49it/s]


Target มีทำ map กลับเป็น int ไว้เลย

In [10]:
lookups = get_lookups(
    config=cfg.lookup,
    data=data,
    label_transform=label_transform,
    text_transform=text_transform,
)

# print data info
pprint(lookups.data_info)
# pprint(lookups.data_info["num_classes"] == len(lookups.code_system2code_indices['icd10_diag'])+len(lookups.code_system2code_indices['icd10_proc']))


#สังเกตแค่่ train เท่านั้นหากเราพยายามเปลียน input เพื่อทดสอบ n น้อยๆ ให้ run model ผ่าน ตอน val / test ถูกล็อคไว้หมดแล้ว

{'average_classes_per_example': 15.65479481182224,
 'average_tokens_per_example': 1596.879217847855,
 'num_classes': 7942,
 'num_examples': 122278,
 'num_test_classes': 7937,
 'num_test_examples': 21265,
 'num_train_classes': 7938,
 'num_train_examples': 76933,
 'num_train_tokens': 122848593,
 'num_val_classes': 7932,
 'num_val_examples': 24080,
 'pad_index': 0,
 'vocab_size': 50001}


ระบุโมเดล

In [11]:
model = get_model(
    config=cfg.model, data_info=lookups.data_info, text_encoder=text_encoder
)
model.to(device)

PLMICD(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50008, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [12]:
# นับจำนวนพารามิเตอร์ทั้งหมด
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# ตรวจสอบจำนวนพารามิเตอร์ในโมเดล
num_parameters = count_parameters(model)
print(f"Total trainable parameters: {num_parameters}")

Total trainable parameters: 137244934


จัดเตรียม datasset

In [ ]:
datasets = get_datasets(
    config=cfg.dataset,
    data=data,
    text_transform=text_transform,
    label_transform=label_transform,
    lookups=lookups,
)

สามารถปรับ batch Optimize ได้ตรงนี้เลย ตรงนี้เป็นการบังคับเฉพาะให้เฉพาะ train เท่านั้น ถูกควบคุมโดย cfg.dataloader.name

set loader มีทั้ง train train_val test val

In [ ]:
#ตรงนี้เตรียม batch ให้เรียบร้อยแล้วหละ เดิม #dataset ยังไม่ทำเป็น batch นะ 
dataloaders = get_dataloaders(config=cfg.dataloader, datasets_dict=datasets)

set optimiser

In [ ]:
optimizer = get_optimizer(config=cfg.optimizer, model=model)
optimizer

เรื่องของ batchsize ตำนวนหา gradient กรณี accumulate_grad_batches > 1 แสดงว่ายังไม่ปรับ grad ทันที & Learning rate

In [ ]:
accumulate_grad_batches = int(
    max(cfg.dataloader.batch_size / cfg.dataloader.max_batch_size, 1)
)
num_training_steps = (
    math.ceil(len(dataloaders["train"]) / accumulate_grad_batches)
    * cfg.trainer.epochs
)
lr_scheduler = get_lr_scheduler(
    config=cfg.lr_scheduler,
    optimizer=optimizer,
    num_training_steps=num_training_steps,
)

split_names: list[str] = ["train", "train_val", "val", "test"],

splits_with_multiple_code_systems: set[str] = {"train_val", "val", "test"},

In [ ]:
#code_system2code_indices => diag proc (ทั้งหมดเลยนะไม่แยก train test)
#split2code_indices =>  train train_val val test
metric_collections = get_metric_collections(
    config=cfg.metrics,
    number_of_classes=lookups.data_info["num_classes"],
    code_system2code_indices=lookups.code_system2code_indices, # รวมทั้งหมดที่เป็น label diag และ label proc
    split2code_indices=lookups.split2code_indices, # label classs แต่ละ กกลุ่มที่แบ่งไป ตาม 
)
metric_collections

#ข้างในมี แต่ index ล้วนๆเลยนะ

- name: WandbCallback

- name: SaveBestModelCallback

- name: EarlyStoppingCallback



In [ ]:
callbacks = get_callbacks(config=cfg.callbacks)
callbacks

Before Train

In [ ]:
eval = Trainer(
    config=cfg,
    data=data,
    model=model,
    optimizer=optimizer,
    dataloaders=dataloaders,
    metric_collections=metric_collections, # metric มี 4 กลุ่ม ตอน ใช้ metric จะแยกกล่มใครกล่มมั่นไปแล้ว
    callbacks=callbacks,
    lr_scheduler=lr_scheduler,
    lookups=lookups,
    accumulate_grad_batches=accumulate_grad_batches,
    experiment_path = Path(cfg.load_model) # ไว้สำหรับ eval 
).to(device)

Eval output จะเก็บไว้ที่ load_model เลย

In [ ]:
if cfg.load_model:
    eval.experiment_path = Path(cfg.load_model)
eval.fit()